## load model

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
with open("hf.token", "r") as f:
    hftoken = f.read().strip()  

import os
cache_dir = "/mnt/c/Users/yc/.cache/huggingface"
os.environ['HF_HOME'] = cache_dir

from huggingface_hub import login
login(token=hftoken)  # Move token to environment variable

from ult import *

/home/yc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
device_map = {"": 0}
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards: 100%|██████████| 4/4 [01:46<00:00, 26.69s/it]


## data

In [ ]:
txt_file_path = 'data/formaldef.txt'
formaldic = txt_to_dict(txt_file_path)
len(formaldic)

meddict={}
for k,v in formaldic.items():
    meddict[k.split('Listen to pronunciation')[0].split('(')[0]]=v
filename= 'data/filtered_medical_dictionary.csv'
eighth_grade_words=set()
with open(filename, 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        if row:  # Make sure row is not empty
            eighth_grade_words.add(row[0])  # Add the word (first column)
filtered_meddict = {word: explanation for word, explanation in meddict.items() 
                   if word in eighth_grade_words}
meddict=filtered_meddict
# load data
df = pd.read_csv('data/CORAL/coral-expert-curated-medical-oncology-reports-to-advance-language-model-inference-1.0/coral/unannotated/data/breastca_unannotated.csv')
# df = df.sample(1, random_state=42)
df=df.iloc[[44,45,53,70,83]] 
test_note=df.iloc[0]['note_text']
df

,coral_idx,Sex,UCSFDerivedRaceEthnicity_X,BirthDate,note_text
44,184,Female,Latinx,1983-10-04,Medical Oncology Consult Note Video Consult ...
45,185,Female,Multi-Race/Ethnicity,1973-03-08,This is a shared service. Physician Statement...
53,193,Female,Multi-Race/Ethnicity,1979-06-20,ID: ***** ***** is a 39 y.o. premenopausal pat...
70,210,Female,Southwest Asian and North African,1974-04-05,Patient Name: ***** ***** ***** *****: 08/22/...
83,223,Female,Unknown/Declined,1960-10-12,We performed this consultation using real-time...


# save

In [ ]:
extraction_prompts = {
    "Reason_for_Visit": """
TASK: Extract 'Reason for Visit'.
Respond *only* with a JSON object using this exact schema:
{
"visit_type": "e.g., New patient, Surveillance, Side effect management",
"summary": "A brief summary of the reason for visit."
}
""",


    "What_We_Found": """
TASK: Extract 'What We Found'.
Respond *only* with a JSON object using this exact schema:
{
"imaging_summary": "Summary of most recent imaging.",
"lab_summary": "Summary of key lab results.",
"performance_status": "e.g., ECOG 0, Stable",
"findings": "Summary of new findings or disease status."
}
""",


    "Treatment_Summary": """
TASK: Extract 'Treatment Summary'.
Respond *only* with a JSON object using this exact schema:
{
"current_meds": "List of current oncologic medications or regimens.",
"recent_changes": "Any holds, dose reductions, or switches.",
"supportive_meds": "List of supportive medications."
}
""",

    "What_We_Discussed": """
TASK: Extract 'What We Discussed / Decided'.
Respond *only* with a JSON object using this exact schema:
{
"goals_of_treatment": "e.g., Palliative, Surveillance",
"response_assessment": "How the cancer is responding.",
"side_effects_discussed": "Side effects discussed and match them to the corresponding medications.",
"other_discussions": "e.g., Clinical trials, palliative care"
}
""",

    "Plan_Going_Forward": """
TASK: Extract 'Plan Going Forward'.
Respond *only* with a JSON object using this exact schema:
{
"medication_plan": "e.g., Continue, Start, Stop",
"imaging_plan": "Which scans and when.",
"lab_plan": "Which labs and when.",
"referrals": "List of any new referrals.",
"follow_up": "When the next visit is."
}
""",

"Medication_Plan": """
TASK: Extract the 'Medication Plan'.
Include future medication plans, changes to current meds (start, stop, continue), supportive meds, bowel regimen, and any blood transfusion plan.
Respond *only* with a JSON object using this exact schema:
{
    "medication_plan": "A summary of the complete medication plan."
}
""",

"Imaging and Lab Plan": """
TASK: Extract the 'Imaging Plan'.
Include all future imaging like CT, MRI, PET, ultrasound,  DEXA scans, including timing if mentioned.
Include future labs like CBC, CMP, tumor markers, coagulation profile, etc. Specify frequency and rationale if mentioned.
Respond *only* with a JSON object using this exact schema:
{
    "imaging_lab_plan": "A summary of the future imaging plan (e.g., 'CT chest in 3 months')."
}
""",

"Genetic_Testing_Plan": """
TASK: Extract the 'Genetic Testing Plan'.
Include any molecular or genetic testing, such as tumor sequencing, germline panels, or liquid biopsy, do not include testings that are already done.
Respond *only* with a JSON object using this exact schema:
{
    "genetic_testing_plan": "A summary of any future planned genetic or molecular tests."
}
""",

"Procedure_Plan": """
TASK: Extract the 'Procedure Plan'.
Include future procedures like surgery, radiation therapy, or interventional procedures.
Respond *only* with a JSON object using this exact schema:
{
    "procedure_plan": "A summary of any planned procedures."
}
""",

"Referral": """
TASK: Extract 'Referral'.
Respond *only* with a JSON object using this exact schema:
{
"Nutrition": "any nutration referrals such as diet optimization, appetite, weight maintenance), "Genetics": "eg, germline testing, family counseling",
"other": "eg, Palliative care (symptom or pain management, goals of care), Radiation oncology / surgical oncology, Physical or occupational therapy, Social work, financial counseling, Psychology / psychiatry for coping and mood support"
}
""",

"follow up/next visit": """
TASK: Extract the 'follow up/next visit'.
Respond *only* with a JSON object using this exact schema:
{
    "Next clinic visit": "(in-person or telehealth): timing and purpose"
}
""",

"Advance care planning": """
TASK: Extract the 'Advance care planning'.
Include Advance directives, health-care proxy, code status (if appropriate)、
Respond *only* with a JSON object using this exact schema:
{
    "Advance care": "A summary of any planned Advance care."
}
""",

}

In [39]:
res=[]

for row in df.itertuples():
    test_note=row.note_text
    gen_config = {
    "max_new_tokens": 512,  # <-- Increased from 150 to 512
    "eos_token_id": tokenizer.eos_token_id,
    "do_sample": False
    }

    # myprint("\n--- 1. Creating Base KV Cache from long text... ---")

    base_prompt = (
        f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
        f"You are a medical data extraction expert. You will be given a long medical note. "
        f"Your task is to answer a series of questions about it, one by one. "
        # FIX 2: Stricter instruction in system prompt
        f"Respond *only* with the valid JSON object requested. Do not add markdown backticks or any other text."
        f"<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
        f"Here is the medical note:\n\n"
        f"--- BEGIN NOTE ---\n{test_note}\n--- END NOTE ---"
        f"\n\nI will now ask you to extract specific sections. "
        f"Please wait for my first extraction task."
        f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        f"{{\"status\": \"Understood. I have read the note and am ready.\"}}"
    )

    # myprint("  Tokenizing and running forward pass to get base cache...")
    with torch.no_grad():
        inputs = tokenizer(base_prompt, return_tensors="pt").to(model.device)
        
        # This logic to create the base_cache is correct
        outputs = model(
            input_ids=inputs["input_ids"], 
            attention_mask=inputs["attention_mask"],
            use_cache=True
        )
        
        base_cache = outputs.past_key_values
        
        # Clean up
        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    # myprint("Base KV Cache created. Ready for 'branching' extractions.")

    # --- FIX 3: Define STRICT SCHEMAS for each extraction task ---
    # This is the most important "prompt engineering" fix.
    # We are telling the model *exactly* what keys to use.


    extracted_data = {}

    # myprint("\n--- 2. Running EFFICIENT 'Branching' Extractions ---")

    # (Assuming `run_model_with_cache_manual` is your function `run_model_with_cache`)
    # If not, please rename this call to `run_model_with_cache`
    run_model_function = run_model_with_cache_manual 

    for key, task in extraction_prompts.items():
        # myprint(f"\nExtracting: {key}...")
        
        # --- FIX 4: Remove the BOS token from the loop ---
        # The base_cache *already* contains the BOS token.
        # Adding it again can cause errors.
        task_prompt = (
            f"<|start_header_id|>user<|end_header_id|>\n\n" # <-- Removed <|begin_of_text|>
            f"{task}" # <-- Use the new, detailed prompt
            f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        )

        # Use the SAME base_cache for each extraction
        answer, returned_cache = run_model_function(
            task_prompt, 
            model, 
            tokenizer, 
            gen_config, 
            kv_cache=base_cache
        )
        
        # Your memory management here is good.
        del returned_cache
        torch.cuda.empty_cache()
        gc.collect()
        
        # myprint(f"  Raw Output: {answer}")
        
        try:
            # We are now *only* expecting a JSON object
            clean_answer = answer.strip().strip("```json").strip("```").strip()
            extracted_data[key] = json.loads(clean_answer)
        except json.JSONDecodeError:
            extracted_data[key] = {"error": "Failed to parse JSON", "raw": answer}

    # myprint("\n--- 3. All extractions complete. ---")
    # myprint("\n--- FINAL EXTRACTED DATA ---")
    res.append(json.dumps(extracted_data, indent=2))

In [ ]:
resdf=[json.loads(r) for r in res]
for i in range(5):
    test_note=df.iloc[i]['note_text']
    resdf[i]['note']=test_note
    resdf[i]['coral_idx']=df.iloc[i].coral_idx
resdf=pd.DataFrame(resdf)
mysave(resdf, base_filename='output/keysummary', extension='.csv')